# Assessment 3

In [1]:
import glob
import os
import numpy as np
import pandas as pd

First we need to import the data. References 1 & 2 introduced me to glob which then allows us to import all the text documents in one smooth process. We do this below.

In [2]:
from pathlib import Path

corpus = []

for path in Path('All enron unzip').rglob('*.txt'):
        with open(path, errors='ignore') as f_input:
            corpus.append(f_input.read())

In [3]:
df = pd.DataFrame(corpus)

In [4]:
df.head()

,0
0,Legitimate\n----------\n- Owner: farmer-d\n- T...
1,Subject: christmas tree farm pictures\n
2,"Subject: vastar resources , inc .\ngary , prod..."
3,Subject: calpine daily gas nomination\n- calpi...
4,Subject: re : issue\nfyi - see note below - al...


In [5]:
print(len(df))

27721


While importing the files, we've also imported the summary sheets along with them. These need to be removed as they wont contribute to our overall corpus and we do this below along with reindexing the data set.

In [6]:
df = df.drop([0,5173,11031,16544,21720]) # removing unnecessary entries
nlist = np.arange(0,27717,1) # reindexing
df = df.reindex(nlist)
df = df.drop(0)

In [7]:
df.head()

,0
1,Subject: christmas tree farm pictures\n
2,"Subject: vastar resources , inc .\ngary , prod..."
3,Subject: calpine daily gas nomination\n- calpi...
4,Subject: re : issue\nfyi - see note below - al...
5,Subject: meter 7268 nov allocation\nfyi .\n- -...


### References

1. https://stackoverflow.com/questions/42407976/loading-multiple-text-files-from-a-folder-into-a-python-list-variable
2. https://stackoverflow.com/questions/2186525/how-to-use-glob-to-find-files-recursively